## Var mapping

In [2]:
from typing import Tuple, Iterable

def var_mapping(r: int, c: int, v: int, N: int) -> int:
    """
    Map (row, column, value) to variable number.

    - r, c, v are 1-based indices
    - N is grid size
    - returns int in 1..N^3
    """
    return (r  * N * N) + (c  * N) + v

## Column constraint

In [3]:
def column_constraint(N: int) -> Iterable[Iterable[int]]:
    """
    Column constraint.

    - For each value v and each column c, exactly one row r has v
    """
    clauses = []
    for r in range(0, N-1):
        for v in range(1, N):
            clause = []
            for c in range(0, N-1):
                for c2 in range(c + 1, N):
                    var1 = var_mapping(r, c, v, N)
                    var2 = var_mapping(r, c2, v, N)
                    clauses.append([-var1, -var2])
            clauses.append(clause)
    return clauses

print(column_constraint(9))

[[-1, -10], [-1, -19], [-1, -28], [-1, -37], [-1, -46], [-1, -55], [-1, -64], [-1, -73], [-10, -19], [-10, -28], [-10, -37], [-10, -46], [-10, -55], [-10, -64], [-10, -73], [-19, -28], [-19, -37], [-19, -46], [-19, -55], [-19, -64], [-19, -73], [-28, -37], [-28, -46], [-28, -55], [-28, -64], [-28, -73], [-37, -46], [-37, -55], [-37, -64], [-37, -73], [-46, -55], [-46, -64], [-46, -73], [-55, -64], [-55, -73], [-64, -73], [], [-2, -11], [-2, -20], [-2, -29], [-2, -38], [-2, -47], [-2, -56], [-2, -65], [-2, -74], [-11, -20], [-11, -29], [-11, -38], [-11, -47], [-11, -56], [-11, -65], [-11, -74], [-20, -29], [-20, -38], [-20, -47], [-20, -56], [-20, -65], [-20, -74], [-29, -38], [-29, -47], [-29, -56], [-29, -65], [-29, -74], [-38, -47], [-38, -56], [-38, -65], [-38, -74], [-47, -56], [-47, -65], [-47, -74], [-56, -65], [-56, -74], [-65, -74], [], [-3, -12], [-3, -21], [-3, -30], [-3, -39], [-3, -48], [-3, -57], [-3, -66], [-3, -75], [-12, -21], [-12, -30], [-12, -39], [-12, -48], [-12, -

## Orthogonal contraint with diff must not be absolute 1

In [19]:
def orthogonal_constraint(N: int) -> Iterable[Iterable[int]]:
    """
    Generate clauses for orthogonal constraints.

    - For every cell (r, c) and each orthogonal neighbor (r′, c′):
      value(r, c) − value(r′, c′)̸ = 1.
    """
    clauses = []
    directions = [(0, 1),(1, 0)]  # check right and down
    for r in range(0, N):
        for c in range(0, N):
            for dr, dc in directions:
                r2, c2 = r + dr, c + dc 
                if 0 <= r2 < N and 0 <= c2 < N:
                    for v in range(1, N):
                        var = var_mapping(r, c, v, N)
                        var_neighborPos = var_mapping(r2, c2, v + 1, N)
                        clauses.append([-var, -var_neighborPos])
                        var_neighborNeg = var_mapping(r2, c2, v - 1, N)
                        clauses.append([-var, -var_neighborNeg])
    return clauses
print(orthogonal_constraint(9))

[[-1, -11], [-1, -9], [-2, -12], [-2, -10], [-3, -13], [-3, -11], [-4, -14], [-4, -12], [-5, -15], [-5, -13], [-6, -16], [-6, -14], [-7, -17], [-7, -15], [-8, -18], [-8, -16], [-1, -83], [-1, -81], [-2, -84], [-2, -82], [-3, -85], [-3, -83], [-4, -86], [-4, -84], [-5, -87], [-5, -85], [-6, -88], [-6, -86], [-7, -89], [-7, -87], [-8, -90], [-8, -88], [-10, -20], [-10, -18], [-11, -21], [-11, -19], [-12, -22], [-12, -20], [-13, -23], [-13, -21], [-14, -24], [-14, -22], [-15, -25], [-15, -23], [-16, -26], [-16, -24], [-17, -27], [-17, -25], [-10, -92], [-10, -90], [-11, -93], [-11, -91], [-12, -94], [-12, -92], [-13, -95], [-13, -93], [-14, -96], [-14, -94], [-15, -97], [-15, -95], [-16, -98], [-16, -96], [-17, -99], [-17, -97], [-19, -29], [-19, -27], [-20, -30], [-20, -28], [-21, -31], [-21, -29], [-22, -32], [-22, -30], [-23, -33], [-23, -31], [-24, -34], [-24, -32], [-25, -35], [-25, -33], [-26, -36], [-26, -34], [-19, -101], [-19, -99], [-20, -102], [-20, -100], [-21, -103], [-21, -1

In [ ]:
def orthogonal_constraint(N: int):
    clauses = []
    # Only right and down neighbors to avoid double encoding
    neighbors = [(0, 1), (1, 0)]
    for r in range(N):
        for c in range(N):
            for dr, dc in neighbors:
                r2, c2 = r + dr, c + dc
                if 0 <= r2 < N and 0 <= c2 < N:
                    # for k=1..N-1: forbid k next to k+1 in both directions
                    for k in range(1, N):
                        clauses.append([-var_mapping(r,  c,  k,   N),-var_mapping(r2, c2, k+1, N)])
                        clauses.append([-var_mapping(r,  c,  k+1, N),-var_mapping(r2, c2, k,   N)])
    print(len(clauses))
    return clauses
print(orthogonal_constraint(9))

2304
[[-1, -11], [-2, -10], [-2, -12], [-3, -11], [-3, -13], [-4, -12], [-4, -14], [-5, -13], [-5, -15], [-6, -14], [-6, -16], [-7, -15], [-7, -17], [-8, -16], [-8, -18], [-9, -17], [-1, -83], [-2, -82], [-2, -84], [-3, -83], [-3, -85], [-4, -84], [-4, -86], [-5, -85], [-5, -87], [-6, -86], [-6, -88], [-7, -87], [-7, -89], [-8, -88], [-8, -90], [-9, -89], [-10, -20], [-11, -19], [-11, -21], [-12, -20], [-12, -22], [-13, -21], [-13, -23], [-14, -22], [-14, -24], [-15, -23], [-15, -25], [-16, -24], [-16, -26], [-17, -25], [-17, -27], [-18, -26], [-10, -92], [-11, -91], [-11, -93], [-12, -92], [-12, -94], [-13, -93], [-13, -95], [-14, -94], [-14, -96], [-15, -95], [-15, -97], [-16, -96], [-16, -98], [-17, -97], [-17, -99], [-18, -98], [-19, -29], [-20, -28], [-20, -30], [-21, -29], [-21, -31], [-22, -30], [-22, -32], [-23, -31], [-23, -33], [-24, -32], [-24, -34], [-25, -33], [-25, -35], [-26, -34], [-26, -36], [-27, -35], [-19, -101], [-20, -100], [-20, -102], [-21, -101], [-21, -103], [

## Orthogonal constraints with diff must not be 1

In [12]:
def orthogonal_constraint(N: int) -> Iterable[Iterable[int]]:
    """
    Generate clauses for orthogonal constraints.

    - For every cell (r, c) and each orthogonal neighbor (r′, c′):
      value(r, c) − value(r′, c′)̸ = 1.
    """
    clauses = []
    directions = [(1, 0), (0, 1)]  # up, down, left, right
    for r in range(0, N ):
        for c in range(0, N ):
            for dr, dc in directions:
                r2, c2 = r + dr, c + dc 
                if 0 <= r2 < N and 0 <= c2 < N:
                    for v in range(1, N):
                        var = var_mapping(r, c, v, N)
                        var_neighborPos = var_mapping(r2, c2, v + 1, N)
                        clauses.append([-var, -var_neighborPos])
    print(len(clauses))
    return clauses
print(orthogonal_constraint(9))

1152
[[-1, -83], [-2, -84], [-3, -85], [-4, -86], [-5, -87], [-6, -88], [-7, -89], [-8, -90], [-1, -11], [-2, -12], [-3, -13], [-4, -14], [-5, -15], [-6, -16], [-7, -17], [-8, -18], [-10, -92], [-11, -93], [-12, -94], [-13, -95], [-14, -96], [-15, -97], [-16, -98], [-17, -99], [-10, -20], [-11, -21], [-12, -22], [-13, -23], [-14, -24], [-15, -25], [-16, -26], [-17, -27], [-19, -101], [-20, -102], [-21, -103], [-22, -104], [-23, -105], [-24, -106], [-25, -107], [-26, -108], [-19, -29], [-20, -30], [-21, -31], [-22, -32], [-23, -33], [-24, -34], [-25, -35], [-26, -36], [-28, -110], [-29, -111], [-30, -112], [-31, -113], [-32, -114], [-33, -115], [-34, -116], [-35, -117], [-28, -38], [-29, -39], [-30, -40], [-31, -41], [-32, -42], [-33, -43], [-34, -44], [-35, -45], [-37, -119], [-38, -120], [-39, -121], [-40, -122], [-41, -123], [-42, -124], [-43, -125], [-44, -126], [-37, -47], [-38, -48], [-39, -49], [-40, -50], [-41, -51], [-42, -52], [-43, -53], [-44, -54], [-46, -128], [-47, -129], 

## Clues

In [8]:
def clues_constraint(sudoku: str) -> Iterable[Iterable[int]]:
    """
    Generate clauses for clues constraints.

    - For each clue in the puzzle, add a unit clause.
    """
    clauses = []
    f = open(sudoku, 'r')
    sudoku = f.read()
    splitlines = sudoku.splitlines()
    N = len(splitlines)
    for r in range(N):
        line = splitlines[r]
        cleanLine = line.replace(" ", "")
        intCleanLine = [int(num) for num in cleanLine]
        # print(f"Integer Line {r}: {(intCleanLine)}")
        for num in range(N):
            v = intCleanLine[num]
            if v != 0:
                c = num
                var = var_mapping(r, c, v, N)
                clauses.append([var])
        # print(clauses)
    print(len(clauses))
    return clauses
clues_constraint('example_n9.txt')

28


[[9],
 [44],
 [61],
 [88],
 [92],
 [140],
 [152],
 [179],
 [195],
 [200],
 [257],
 [269],
 [272],
 [303],
 [372],
 [398],
 [421],
 [425],
 [436],
 [450],
 [473],
 [490],
 [552],
 [575],
 [579],
 [680],
 [701],
 [727]]

In [23]:
def at_least_one(N):
    """
    Exactly one value per cell

    - For each cell (r, c), exactly one v ∈ {1, . . . , N } is true.
    """
    clauses = list()
    clause = list()

    for r in range(N):
        for c in range(N):
            for v in range(1, N + 1):
                literal = var_mapping(r, c, v, N)
                clause.append(literal)
            clauses.append(clause)
            clause = list()
    print(len(clauses), "clauses for at least one")
    return clauses


def at_most_one(N):
    clauses = list()
    for r in range(N):
        for c in range(N):
            # Check first for only cell 0,0
            for v1 in range(1, N):
                for v2 in range(v1 + 1, N + 1):
                    clause_single = [var_mapping(r,c,v1,N) * -1, var_mapping(r,c,v2,N) * -1]
                    clauses.append(clause_single)
    print(len(clauses), "clauses for at most one")
    return clauses

def exactly_one_v_per_cel(N):
    # Both at most one AND at least one are true
    clauses = at_least_one(N)
    clauses.extend(at_most_one(N))
    return clauses

print(len(exactly_one_v_per_cel(9)))
print("The length for exactly one value per cell should be 2916 81*36 = 2916")

81 clauses for at least one
2916 clauses for at most one
2997
The length for exactly one value per cell should be 2916 81*36 = 2916


In [ ]:
def row_constraint(N: int) -> Iterable[Iterable[int]]:
    """
    Generate clauses for row constraints.

    - For each value v and each row r, exactly one column c has v
    """
    clauses = list()
    for r in range(N):
        for v in range(1, N + 1):
            clauses.append([var_mapping(r,c,v,N) for c in range(N)])  # at least one clause
            for c1 in range(N - 1):
                for c2 in range(c1 + 1, N):
                    clause_single = [var_mapping(r,c1,v,N) * -1,
                                     var_mapping(r,c2,v,N) * -1]
                    clauses.append(clause_single)
    print(len(clauses), "row constraints clauses")
    return clauses
row_constraint(9)

2916 row constraints clauses


[[-1, -10],
 [-1, -19],
 [-1, -28],
 [-1, -37],
 [-1, -46],
 [-1, -55],
 [-1, -64],
 [-1, -73],
 [-10, -19],
 [-10, -28],
 [-10, -37],
 [-10, -46],
 [-10, -55],
 [-10, -64],
 [-10, -73],
 [-19, -28],
 [-19, -37],
 [-19, -46],
 [-19, -55],
 [-19, -64],
 [-19, -73],
 [-28, -37],
 [-28, -46],
 [-28, -55],
 [-28, -64],
 [-28, -73],
 [-37, -46],
 [-37, -55],
 [-37, -64],
 [-37, -73],
 [-46, -55],
 [-46, -64],
 [-46, -73],
 [-55, -64],
 [-55, -73],
 [-64, -73],
 [-2, -11],
 [-2, -20],
 [-2, -29],
 [-2, -38],
 [-2, -47],
 [-2, -56],
 [-2, -65],
 [-2, -74],
 [-11, -20],
 [-11, -29],
 [-11, -38],
 [-11, -47],
 [-11, -56],
 [-11, -65],
 [-11, -74],
 [-20, -29],
 [-20, -38],
 [-20, -47],
 [-20, -56],
 [-20, -65],
 [-20, -74],
 [-29, -38],
 [-29, -47],
 [-29, -56],
 [-29, -65],
 [-29, -74],
 [-38, -47],
 [-38, -56],
 [-38, -65],
 [-38, -74],
 [-47, -56],
 [-47, -65],
 [-47, -74],
 [-56, -65],
 [-56, -74],
 [-65, -74],
 [-3, -12],
 [-3, -21],
 [-3, -30],
 [-3, -39],
 [-3, -48],
 [-3, -57],
 [-3, -6